In [1]:
# Load the imports
# import keras
import numpy as np
import pandas as pd
# import plotly.express as px

# Adjusts the granularity of reporting
pd.options.display.float_format = "{:.1f}".format

In [2]:
# Load and read the datasets
df_generics = pd.read_csv("../data/raw/df_generics_train.csv")
df_info = pd.read_csv("../data/raw/df_medicine_info_train.csv")
df_volume = pd.read_csv("../data/raw/df_volume_train.csv")
df_generics_test = pd.read_csv("../data/raw/df_generics_test.csv")
df_info_test = pd.read_csv("../data/raw/df_medicine_info_test.csv")
df_volume_test = pd.read_csv("../data/raw/df_volume_test.csv")

print(df_generics.shape)
display(df_generics.head())
display(df_generics.describe())

print(df_info.shape)
display(df_info.head())
display(df_info.describe())

print(df_volume.shape)
display(df_volume.head())
display(df_volume.describe())

print(df_generics_test.shape)
display(df_generics_test.head())
display(df_generics_test.describe())

print(df_info_test.shape)
display(df_info_test.head())
display(df_info_test.describe())

print(df_volume_test.shape)
display(df_volume_test.head())
display(df_volume_test.describe())

(46872, 4)


,country,brand_name,months_postgx,n_gxs
0,COUNTRY_B6AE,BRAND_1C1E,0,4.0
1,COUNTRY_B6AE,BRAND_1C1E,1,4.0
2,COUNTRY_B6AE,BRAND_1C1E,2,4.0
3,COUNTRY_B6AE,BRAND_1C1E,3,4.0
4,COUNTRY_B6AE,BRAND_1C1E,4,4.0


,months_postgx,n_gxs
count,46872.0,34536.0
mean,11.5,3.8
std,6.9,4.6
min,0.0,0.0
25%,5.8,1.0
50%,11.5,2.0
75%,17.2,5.0
max,23.0,43.0


(1953, 7)


,country,brand_name,ther_area,hospital_rate,main_package,biological,small_molecule
0,COUNTRY_0024,BRAND_1143,Sensory_organs,0.1,EYE DROP,False,True
1,COUNTRY_0024,BRAND_1865,Muscoskeletal_Rheumatology_and_Osteology,92.4,INJECTION,False,False
2,COUNTRY_0024,BRAND_240F,Antineoplastic_and_immunology,36.9,PILL,False,True
3,COUNTRY_0024,BRAND_2F6C,Antineoplastic_and_immunology,0.0,INJECTION,True,False
4,COUNTRY_0024,BRAND_3A67,Nervous_system,NaN,PILL,False,False


...

,country,brand_name,ther_area,hospital_rate,main_package,biological,small_molecule
0,COUNTRY_0024,BRAND_31BE,Nervous_system,0.3,PILL,False,True
1,COUNTRY_0024,BRAND_5165,Anti-infectives,4.3,PILL,False,True
2,COUNTRY_0024,BRAND_79B0,Nervous_system,0.7,PILL,False,True
3,COUNTRY_0024,BRAND_8164,Cardiovascular_Metabolic,1.3,PILL,False,True
4,COUNTRY_01A1,BRAND_03DE,Muscoskeletal_Rheumatology_and_Osteology,1.5,PILL,False,True


,hospital_rate
count,340.0
mean,29.8
std,38.5
min,0.0
25%,1.0
50%,6.4
75%,61.6
max,100.0


(8832, 5)


,country,brand_name,month,months_postgx,volume
0,COUNTRY_9891,BRAND_DB48,Oct,-24,231503.8
1,COUNTRY_9891,BRAND_DB48,Nov,-23,203754.7
2,COUNTRY_9891,BRAND_DB48,Dec,-22,224900.0
3,COUNTRY_9891,BRAND_DB48,Jan,-21,252599.9
4,COUNTRY_9891,BRAND_DB48,Feb,-20,229358.6


,months_postgx,volume
count,8832.0,8832.0
mean,-11.4,3118819.9
std,7.8,10884275.1
min,-24.0,38.4
25%,-18.0,19877.7
50%,-12.0,174353.6
75%,-5.0,1240780.2
max,5.0,161927296.2


In [3]:
# Find most fequent values for imputation
most_frequent = df_info_test.mode().iloc[0]
print(most_frequent)

country             COUNTRY_F75D
brand_name            BRAND_3CB9
ther_area         Nervous_system
hospital_rate                0.3
main_package                PILL
biological                 False
small_molecule              True
Name: 0, dtype: object


In [4]:
# Merge datasets
df_merged_1 = pd.merge(
    df_generics,
    df_info,
    on=["country", "brand_name"],
    how="left"
)

df_final = pd.merge(
    df_volume,
    df_merged_1,
    on=["country", "brand_name", "months_postgx"],
    how="left"
)

df_merged_1_test = pd.merge(
    df_generics_test,
    df_info_test,
    on=["country", "brand_name"],
    how="left"
)

df_final_test = pd.merge(
    df_volume_test,
    df_merged_1_test,
    on=["country", "brand_name", "months_postgx"],
    how="left"
)

In [5]:
# Fill missing values from other columns
cols_to_fill = [
    "n_gxs",
    "ther_area",
    "hospital_rate",
    "main_package",
    "biological",
    "small_molecule",
]

ref = (
    df_final[df_final["months_postgx"] == 0]
    .set_index(["country", "brand_name"])[cols_to_fill]
)

ref_test = (
    df_final_test[df_final_test["months_postgx"] == 0]
    .set_index(["country", "brand_name"])[cols_to_fill]
)

df_with_ref = df_final.join(
    ref,
    on=["country", "brand_name"],
    rsuffix="_ref"
)

df_with_ref_test = df_final_test.join(
    ref_test,
    on=["country", "brand_name"],
    rsuffix="_ref"
)

for col in cols_to_fill:
    ref_col = col + "_ref"
    df_with_ref[col] = df_with_ref[col].fillna(df_with_ref[ref_col])
    df_with_ref_test[col] = df_with_ref_test[col].fillna(df_with_ref_test[ref_col])

# Drop the helper columns
df_with_ref = df_with_ref.drop(columns=[c + "_ref" for c in cols_to_fill])
df_with_ref_test = df_with_ref_test.drop(columns=[c + "_ref" for c in cols_to_fill])

C:\Users\harri\AppData\Local\Temp\ipykernel_36960\626460565.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_with_ref[col] = df_with_ref[col].fillna(df_with_ref[ref_col])


In [6]:
df_final = df_with_ref
df_final_test = df_with_ref_test

# Fill missing hospital rate values with mean
df_final["hospital_rate"] = df_final["hospital_rate"].fillna(30.0)
df_final_test["hospital_rate"] = df_final_test["hospital_rate"].fillna(30.0)

# Fill missing number of generics values with 0.0
df_final["n_gxs"] = df_final["n_gxs"].fillna(0.0)
df_final_test["n_gxs"] = df_final_test["n_gxs"].fillna(0.0)

# Set n_gxs to 0.0 where months_postgx < 0.0
df_final.loc[df_final["months_postgx"] < 0.0, "n_gxs"] = 0.0
df_final_test.loc[df_final_test["months_postgx"] < 0.0, "n_gxs"] = 0.0


In [7]:
# Combine train + test with a flag
df_final = df_final.copy()
df_final_test = df_final_test.copy()

df_final["is_train"] = 1
df_final_test["is_train"] = 0
df_all = pd.concat([df_final, df_final_test], ignore_index=True)

In [8]:

# Compute average volumes
mask = (df_all["months_postgx"] >= -12) & (df_all["months_postgx"] <= -1)

avg_vol_lookup = (
    df_all[mask]
    .groupby(["country", "brand_name"])["volume"]
    .mean()
)

df_all["avg_vol"] = df_all.set_index(["country", "brand_name"]).index.map(avg_vol_lookup)

# Normalized volume
post_mask = (df_all["months_postgx"] >= 0) & (df_all["months_postgx"] <= 23)
df_all["vol_norm"] = df_all["volume"] / df_all["avg_vol"]

mean_erosion_lookup = (
    df_all[post_mask]
    .groupby(["country", "brand_name"])["vol_norm"]
    .mean()
)

df_all["mean_generic_erosion"] = (
    df_all.set_index(["country", "brand_name"]).index.map(mean_erosion_lookup)
)

# Create buckets based on mean_generic_erosion
df_all["bucket"] = np.nan
mge = df_all["mean_generic_erosion"]
df_all.loc[mge.between(0.0, 0.25, inclusive="both"), "bucket"] = 1
df_all.loc[mge > 0.25, "bucket"] = 2
df_all["bucket"] = df_all["bucket"].astype("Int64")

# Fill missing mean_generic_erosion values with mean
df_all["mean_generic_erosion"] = df_all["mean_generic_erosion"].fillna(df_all["mean_generic_erosion"].mean())

# Further imputation
df_all["bucket"] = df_all["bucket"].fillna(2)
df_all["ther_area"] = df_all["ther_area"].fillna("Nervous_system")
df_all["main_package"] = df_all["main_package"].fillna("PILL")
df_all["biological"] = df_all["biological"].fillna("False")
df_all["small_molecule"] = df_all["small_molecule"].fillna("True")

In [9]:
# Split back into train and test
df_final = df_all[df_all["is_train"] == 1]
df_final_test  = df_all[df_all["is_train"] == 0]

In [10]:
# Save as .csv files
df_final.to_csv("../data/processed/df_train_merged_filled.csv", index=False)
df_final_test.to_csv("../data/processed/df_test_merged_filled.csv", index=False)